In [1]:
from cuml.ensemble import RandomForestClassifier as cuRF

import cuml
import numpy as np
import cupy as cp
import time
import os
import json


In [2]:
# Cargar el dataset desde el archivo .npy
path1 = '../DatasetVGG19/test.npy'
path2 = '../DatasetVGG19/train.npy'
path3 = '../DatasetVGG19/validation.npy'

data_train = np.load(path2)
data_test = np.load(path1)
data_val = np.load(path3)

In [3]:
def unpack(array):
    x = cp.asarray(array[:,:-2])
    y = array[:,-2:]
    y = cp.asarray(np.argmax(y, axis=1))
    return x,y

def writeJson(tiempo,accu,metod):
    nuevo_registro = {
        'Tipo Ejecucion' : "GPU",
        'Metodo' : metod,
        'tiempo_ejecucion': tiempo,
        'accuracy' : accu
    }
    nombre_archivo = '../GPU_tiempos.json'

    # Leer el contenido existente, si el archivo ya existe
    if os.path.exists(nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            try:
                registros = json.load(archivo)
            except json.JSONDecodeError:
                registros = []
    else:
        registros = []

    # Agregar el nuevo registro
    registros.append(nuevo_registro)

    # Guardar los registros actualizados en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
        json.dump(registros, archivo, indent=4)

In [4]:
X_train,y_train = unpack(data_train)
X_test,y_test = unpack(data_train)
X_val,y_val = unpack(data_val)

In [7]:
def random_forest(X, y):
    # Random Forest con parámetros ajustados
    model = cuRF(n_estimators=100, max_depth=50, random_state=42)
    model.fit(X, y)
    return model

def model_score(X, y, model, dataset):
    accuracy = model.score(X, y)
    print(f"Precisión del modelo {dataset}: {accuracy * 100:.2f}%")
    return accuracy


In [18]:
print("Entrenamiento Random Forest")

inicio = time.time()

RF_model = random_forest(X_train, y_train)

fin = time.time()
tiempo_ejecucion_RF = fin - inicio

accuracy = model_score(X_test, y_test, RF_model, "Test")
model_score(X_val, y_val, RF_model, "Val")

print(f"El tiempo de ejecución fue de {tiempo_ejecucion_RF} segundos")

Entrenamiento Random Forest
Precisión del modelo Test: 86.84%
Precisión del modelo Val: 37.50%
El tiempo de ejecución fue de 4.529031991958618 segundos


In [19]:
writeJson(tiempo_ejecucion_RF, accuracy, "Random Forest")